## Обогащение данных фондового рынка с помощью Open AI API

<p align="center">
    <img src="images/nasdaq100.png" width="450">
</p>

Nasdaq-100 — это индекс фондового рынка, состоящий из 101 долевой ценной бумаги, выпущенной 100 крупнейшими нефинансовыми компаниями, акции которых котируются на фондовой бирже Nasdaq. Это помогает инвесторам сравнивать цены на акции с предыдущими ценами, чтобы определить эффективность рынка.

В этом проекте нам предоставляются два файла CSV, содержащие информацию об акциях Nasdaq-100:
- _**nasdaq100.csv**_: содержит информацию о компаниях в индексе, такую как символ, название и т. д.
- _n**asdaq100_price_change.csv**_: содержит изменения цен на акции за периоды, включая (но не ограничиваясь) один день, пять дней, один месяц, шесть месяцев, один год и т. д.

Мы будем использовать API OpenAI для классификации компаний по секторам и подготовки сводных данных о деятельности секторов и компаний за этот год (2023)

# CSV с данными акций Nasdaq-100

Доступны два файла CSV: `nasdaq100.csv` и `nasdaq100_price_change.csv`.

## nasdaq100.csv

```py
symbol,name,headQuarter,dateFirstAdded,cik,founded
AAPL,Apple Inc.,"Cupertino, CA",,0000320193,1976-04-01
ABNB,Airbnb,"San Francisco, CA",,0001559720,2008-08-01
ADBE,Adobe Inc.,"San Jose, CA",,0000796343,1982-12-01
ADI,Analog Devices,"Wilmington, MA",,0000006281,1965-01-01
...
```

## nasdaq100_price_change.csv

```py
symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
AAPL,-1.7254,-8.30086,-6.20411,3.042,15.64824,42.99992,8.47941,60.96299,245.42031,976.99441,139245.53954
ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.64013,-1.04347,-1.04347,-1.04347,-1.04347
ADBE,0.5409,-1.77817,9.16191,52.0465,38.01522,57.22723,21.96206,17.83037,109.05718,1024.69214,251030.66399
ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.09735,63.42847,92.81874,286.77518,26012.63736
...
```

In [ ]:
# Импорт необходимых модулей
from tenacity import retry, stop_after_attempt, wait_random_exponential
import pandas as pd
import openai


# Инициализация ключа пользователя API
openai.api_key = ["KEY"]

# Определение функции для предотвращения ошибки ограничения скорости (из-за выполнения более одного длинного запроса)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(4))
def backoff_comp(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

# Чтение двух наборов данных
nasdaq100 = pd.read_csv("nasdaq100.csv")
price_change = pd.read_csv("nasdaq100_price_change.csv")

# Добавление 'symbol' в nasdaq100
nasdaq100 = nasdaq100.merge(price_change[["symbol", "ytd"]], on="symbol", how="inner")

# Просмотр объединенного набора данных
nasdaq100.head()

# Создание промпта на обогащение nasdaq100 с помощью OpenAI
prompt = '''Classify company {company} into one of the following sectors. Answer only with the sector name:
    Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, Communication, Energy, Consumer Defensive, Real Estate, Financial.'''

# Просматривая компании NASDAQ
for company in nasdaq100["symbol"]:
    # Создание ответа от ChatGPT с помощью функции 'backoff'
    response = backoff_comp(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt.format(company=company)}],
        temperature=0.0,
    )
    # Сохранение вывода в виде переменной под названием 'sector'
    sector = response['choices'][0]['message']['content']
    
    # Добавление 'sector' для соответствующей компании
    nasdaq100.loc[nasdaq100["symbol"] == company, "Sector"] = sector
    
# Подсчет количества 'sector'
nasdaq100["Sector"].value_counts()

# Промпт для получения рекомендаций по акциям
prompt = '''Provide summary information about Nasdaq-100 stock performance year to date (YTD), 
            recommending the three best sectors and three or more companies per sector.
            Company data: {company_data} '''

# Получение ответа модели
response = backoff_comp(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt.format(company_data=nasdaq100)}],
        temperature=0.0,
    )

# Сохранение вывода в виде переменной и печать рекомендаций
stock_recommendations = response['choices'][0]['message']['content']
print(stock_recommendations)